In [1]:
import re
import urllib
import urllib.parse
import urllib.request

from IPython.display import Markdown
from bs4 import BeautifulSoup

(sec:crawler)=
# 簡易クローラの作成

**クローラ** (crawler)とは、複数のウェブページを巡回しながら情報を収集するプログラムを指す。本節では、日本語版のWikipediaを対象に、記事のリンク関係を調べるクローラを作成してみよう。

## 本クローラの目的

読者の皆さんは、人と人との関係を表すとき、知り合いを6人たどると、おおよそ世界中のどんな人にもたどり着ける、という話を聞いたことがあるだろうか。この話は「六次の隔たり」と呼ばれ、Wikipediaの記事としてもまとめられている。

- [六次の隔たり - Wikipedia](https://ja.wikipedia.org/wiki/%E5%85%AD%E6%AC%A1%E3%81%AE%E9%9A%94%E3%81%9F%E3%82%8A)

この話と同様のことを、ウィキペディアの英語ページに対して行えるウェブサイトに以下のようなものがある。いくつかの記事名を入力して試してみると、すぐに思いつくような記事名であれば、おおむね3回くらいリンクを辿ると一方の記事からもう一方の記事までたどり着けるだろう。

- <https://www.sixdegreesofwikipedia.com/>

なお、このウェブサイトのソースコードはGitHub上に[公開](https://github.com/jwngr/sdow)されており、クローリング済みのデータを使用して経路を計算しているため、動作が非常に速い。実際にHTTP通信を行ないながら経路を探す場合には、どのくらいの時間がかかるのか。ぜひ以下の課題を通して体験してみてほしい。

本項では、日本語版のWikipediaを対象に「2つの記事が何次の隔たりにあるか」を調べるプログラムを作成する。実装するクローラのプログラムは**目安として100行前後**になる。

## ステップ0: スクリプトの作成

今回は独立したスクリプトファイルとしてクローラを作成するので、まずは`crawler.py`のような名前でPythonスクリプトを作成しよう。

スクリプトファイルが作成できたら、まずは、以下のように記述して、コマンドラインから実行した際にプログラム中から`main`関数が呼び出されるようにしておく。

```python
def main():
    # 空のメイン関数
    pass

if __name__ == "__main__":
    main()
```

次に、コマンドライン引数から、スタートの記事名とゴールの記事名を指定できるようにしておく。これには`sys`モジュールの`argv`を用いる。

```python
# 追加
import sys

def main():
    # 追加
    argc = len(sys.argv)


`sys.argv`は最低で長さが1となっており、`sys.argv[0]`には実行したスクリプトファイル名が入る。以降、`sys.argv[1]`、`sys.argv[2]`、...には、コマンドライン引数が順に格納される。

例えば、シェル環境から

```shell
python crawler.py arg1 arg2
```

のようにプログラムを実行すると、`sys.argv[0]`が`crawler.py`、`sys.argv[1]`が`arg1`、`sys.argv[2]`が`arg2`というようになる。

今回のクローラはスタートとゴールの記事名を必ず引数に取るので、`argc = len(sys.argv)`は必ず3でなければならない。従って、コマンドライン引数の数が異なる場合には、エラーメッセージを出力しておく。

```python
def main():
    argc = len(sys.argv)
    # 追加
    if argc != 3:
        print(f"[USAGE] python {sys.argv[0]} START GOAL")
        sys.exit(0)

    start = sys.argv[1]
    goal = sys.argv[2]

## ステップ1: 記事内容の取得

日本語のWikipediaの記事は `https://ja.wikipedia.org/wiki/[記事名]` というURLを持っている。記事名のところにはURLエンコードされた記事名が入る。

例えば、「日本」というWikipediaの記事は、以下のように取得できる。

In [2]:
# Wikipedia - 日本
base_url = "https://ja.wikipedia.org/wiki/"
title = "日本"
url = base_url + urllib.parse.quote(title)
print(f"URL: {url:s}")

URL: https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC


HTMLページの取得方法については、前節、[HTTP通信](sec:request)で説明したとおりである。

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ja",
    "Accept-Charset": "utf-8",
}

req = urllib.request.Request(url, headers=headers, method="GET")
with urllib.request.urlopen(req) as resp:
    body = resp.read().decode("utf-8")

In [4]:
tree = BeautifulSoup(body)
lines = re.split("[\n\r]{1,2}", tree.prettify())
Markdown("```html\n" + "\n".join(lines[:8]) + "\n```")

```html
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="ja">
 <head>
  <meta charset="utf-8"/>
  <title>
   日本 - Wikipedia
  </title>
  <script>
```

ページの内容が取得できたら、[正規表現](sec:regex)を使用して、リンクされている**Wikipedia日本語版の記事**を取得する (プログラムは自分で考えること)。

In [5]:
p_link = re.compile(r"href=\"/wiki/([^:\#\"]*?)\"")
links = p_link.findall(body)
links = list(set(links))
print("\n".join(links[:10]))

1947%E5%B9%B4
%E5%B9%B9%E7%B4%B0%E8%83%9E
%E5%8C%97%E6%97%A5%E6%9C%AC
%E6%AD%BB%E5%9B%A0
%E7%94%9F%E9%BA%A6%E4%BA%8B%E4%BB%B6
%E3%83%9D%E3%83%84%E3%83%80%E3%83%A0%E5%AE%A3%E8%A8%80
2016%E5%B9%B4
%E6%AD%A6%E8%A1%93
%E3%83%AF%E3%82%B7%E3%83%B3%E3%83%88%E3%83%B3%E4%BC%9A%E8%AD%B0_(1922%E5%B9%B4)
%E5%A5%B4%E9%9A%B7


ここで得られる記事名はURLエンコードされているので、`urllib.parse.unquote`関数を用いて、元の日本語の記事名を復元しておく。

In [6]:
titles = [urllib.parse.unquote(link) for link in links]
print("\n".join(titles[:10]))

1947年
幹細胞
北日本
死因
生麦事件
ポツダム宣言
2016年
武術
ワシントン会議_(1922年)
奴隷


:::{admonition} 記事内容の取得
:class: note

ここまでの内容を元に、1つの記事名`title`を引数に取り、その記事にリンクされている、全ての記事名をリストとして返す関数`get_pages`を作成しよう。

```python
def get_pages(title):
    """
    指定されたタイトルのページからリンクされているページのタイトルを取得
    """
    # 以下にコードを実装
    ...
```
:::

## ステップ2: 記事を幅優先探索

幅優先探索それ自体についてはWikipedia等、ウェブ上の記事を参考にしてほしい。Wikipediaの該当記事にはPython実装のソースコードも示されている。

[幅優先探索 - Wikipedia](https://ja.wikipedia.org/wiki/%E5%B9%85%E5%84%AA%E5%85%88%E6%8E%A2%E7%B4%A2)

以下では、標準的なFIFO (first-in-first-out)型のキューを用いた実装を用いる。Pythonでは`queue`モジュールの`SimpleQueue`が使用できる。

In [7]:
from queue import SimpleQueue

幅優先探索による記事探索のアルゴリズムの概要は以下のようになる。

```python
# キューは現在の記事とリンク元の記事を管理
que = SimpleQueue()
que.put((start, None))
checked = {}  # チェック済みなら、記事名をキー、リンク元の記事を値とする要素を持つ

# キューが空になるまで処理を継続
while not que.empty():
    # スロットリング
    time.sleep(0.1)

    # ノードの取り出し
    node = que.get()
    page, prev = node.title

    # 既にチェック済みの記事かどうかを調べる
    # !!! 実装する !!!

    # 未チェックの記事なら、リンクされた記事を取得
    next_pages = get_pages(title)

    # 新しい記事の中にゴールが見つかったら終了
    # !!! 実装する !!!

    # 取得した記事名をキューに追加
    # !!! もう少し早くできる !!!
    for next_page in next_pages:
        que.put((next_page, page))

上記のように、幅優先探索では、まずキューに開始点となるノードを追加する。その後、順次、キューの先頭からノードを取り出して、そのノードと隣接するノードを調べ、キューに追加していく。

今回のクローラの例では、最後に、スタートからゴールの記事へと遷移するリンクを復元したいので、キューには記事名と、リンク元の記事を追加しておくと良い。その上で、辞書型の`checked`にチェック済みの記事を記録しておく。`checked`は記事名をキーに取り、その記事がチェック済みなら、リンク元の記事名を値として持つようにしておけば良いだろう。

なお、上記の疑似コードでは、新しく見つかった記事について、単純に全ての記事をキューに追加しているが、この部分はもう少し高速化できるので、各自考えてみてほしい。

:::{admonition} 記事の幅優先探索
:class: note

上記の内容を元に`main`関数内に幅優先探索のプログラムを実装しよう。実装の内容については、前述の疑似コードの`!!! ... !!!`の部分に適切なコードを与えていけば良い。
:::

:::{admonition} スロットリング (throttling)
:class: warning

クローラのプログラムを作成する際、一つのウェブサーバに何度もリクエストを送ると、ウェブサーバの処理が低下するなどの問題が発生する。従って、クローリングのマナーとして、リクエストを頻繁に送りすぎないよう、待ち時間を設定しなければならない。

このように意図的にネットワークの速度を下げる処理を**スロットリング**と呼び、上記の疑似コードにおいては、リクエストが0.1秒に1回以上送られないように制御している。
:::

## ステップ3: リンクを復元

スタート (`start`)から、幅優先探索でゴール (`goal`)にたどり着いたら、`checked`に保存された、リンク元の記事の情報を使って、スタートからゴールに至る記事の遷移を復元しよう。

`checked`は「現在の記事」をキーとして、「リンク元の記事」を値に持つ辞書型であったので、`goal`の記事から逆順にリンク元の記事を辿っていき、`start`のリンク元である`None`にたどり着いたら、経路が復元できたことになる。

ただし、この処理を単純に実行すると、経路が逆順になるので、スタートからゴールに至る経路を得るには、最後に配列を`reversed`等で逆順にする必要があることに注意すること。

```python
# 経路の取り出し
path = [goal]
# !!! 実装する !!!

# 結果の表示
print("")
print("Answer:")
print(" -> ".join(path))
````


## ステップ4: HTTP通信の高速化

ここまでプログラムを書き進めると、一応のクローラが完成する。しかし、実際に実行してみると、スロットリングの時間を考慮しても、HTTP通信に少々時間がかかっていることに気がつくかもしれない。

これは、`urllib`を用いて送信されるリクエストが、接続と切断を繰り返すためであり、今回のクローラがおおよそ同一と考えられるWikipediaのサーバーに接続・切断を繰り返すのは効率が良いとは言えない (実際には、負荷分散されているため物理的に同じサーバーに接続するわけではない)。

この問題を防ぐには、`urllib`の代わりに、より低レベルな`http.client`モジュールを用いる必要がある。`http.client`モジュールには`HTTPConnection`と`HTTPSConnection`というクラスが用意されており、これらのクラスは指定されたURLに接続後、明示的に`close`関数が呼ばれるか、通信のエラーが発生しない限りは接続が切られることがない。

以下に`http.client`モジュールを用いたHTTPリクエストの送信とレスポンスの処理の一例を示すので、これを参考にHTTP通信の高速化を試みてほしい。

In [8]:
import http.client

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ja",
    "Accept-Charset": "utf-8",
}

# サーバに接続
conn = http.client.HTTPSConnection("ja.wikipedia.org")

# 一つ目の記事の読み込み
title1 = "日本"
conn.request("GET", "/wiki/" + urllib.parse.quote(title1), headers=headers)
resp1 = conn.getresponse()
print(resp1.status, resp1.reason)
data1 = resp1.read().decode("utf-8")
resp1.close()

# 二つ目の記事の読み込み
title2 = "Python"
conn.request("GET", "/wiki/" + urllib.parse.quote(title2), headers=headers)
resp2 = conn.getresponse()
print(resp2.status, resp1.reason)
data2 = resp2.read().decode("utf-8")
resp2.close()

# 切断
conn.close()

200 OK
200 OK


## 課題



:::{card}
**問1**
^^^
本節の内容に従い、Wikipedia日本語版のクローラを作成せよ。クローラが作成できたら、いくつかの記事のペア(3例程度)について、各記事ペアを結ぶ記事の遷移を調べよ。

例1:
```shell
python crawler.py "伊藤博文" "紫式部"
Answer: 伊藤博文 -> 日本銀行券 -> 紫式部
```

例2:
```shell
python crawler.py "ドラえもん" "東京スカイツリー"
Answer: ドラえもん -> 月刊コロコロコミック -> 東京スカイツリー
```
:::

:::{card}
**問2**
^^^
作成したクローラを改良し、適当な単語から3次以上の隔たりを持つ記事を探しなさい。実装にあたっては、スタートの記事からの距離もキューに追加するようにし、ゴールに存在しない記事名 (例えば ```__DUMMY_PAGE__```など)を与えるようにすると、実装が容易になるだろう。

ただし、リンクがつながった記事の数が指数関数的に増えていくため、3次以上の隔たりを持つ記事を探すには相当時間がかかるので注意すること。

なお、ここでの「3次の隔たり」とは、仮にスタートの単語を「日本」とすると、

```text
日本 -> 記事A -> 記事B -> 記事C
```

の「記事C」に相当するものとする。

例1: 「渋沢栄一」からスタート
```text
渋沢栄一 -> 火星 -> 火星探査 -> 地球 
```

例2: 「清少納言」からスタート
```text
清少納言 -> 百人一首 -> 手事物 -> 千鳥の曲
```
:::